In [ ]:
"""
Calculate the micro-AUROC from each fold plus the remaining NSR, and then average them.
Copied from:
/mnt/r/ENGR_Chon/Dong/Github_private/Pulsewatch_labeling/DeepBeat/experiments/try_07_eval_models/main_02_fold_1_2.py
"""
import os
import torch
import pandas as pd
import sys
import itertools
import pathlib
import numpy as np
# === Start of path and model name for all models. ===
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!
import warnings
warnings.filterwarnings("ignore") # I only add this for pandas dataframe groupby warning. You should remove it in the future. 
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!
# For 1D PPG aug5k:
# path_ckpt = r'/mnt/r/ENGR_Chon/Dong/Python_generated_results/deep_learning_2023/y_pred_2024_07_29_aug5k/TestRNNGRU_aug5k_PPG_HR_ACC_rescaleHR'
# file_date = '20240728'
# model_name = '1D_PPG_aug5k_HR_ACC_rescaleHR'
sys.path.append('/mnt/r/ENGR_Chon/Dong/Github_private/Pulsewatch_labeling/DeepBeat/experiments/try_07_eval_models')
import my_model_names
# list_str_model_idx = ['01','02','03','04','05','06','07','08','09','10','11','12','18','19','20','21','22','23','24']
list_str_model_idx = ['03','04','05','06','07','08','09','10','11','12','18','19','20','21','22','23','24']
for str_model_idx in list_str_model_idx:
# if True:
    # str_model_idx = '10'
    print('str_model_idx',str_model_idx)
    path_ckpt, file_date, model_name = my_model_names.my_model_names(str_model_idx)
    # === End of path and model names. ====

    flag_linux = True
    flag_Colab = False
    # Append the directory to your python path using sys
    if flag_linux:
        print('Inside Linux')
        if flag_Colab:
            print('Inside Colab')
            # For 'my_pathdef'
        else:
            print('Not inside Colab')
            # For 'my_cfmatrix'
            sys.path.append('/mnt/r/ENGR_Chon/Dong/Github_private/Pulsewatch_labeling/DeepBeat/utils')
            import pretty_errors

    import my_cfmatrix

    # ====
    if str_model_idx == '01' or str_model_idx == '02':
        # TensorFlow model, so output saved in csv format.
        if str_model_idx == '01':
            filename_fold_1 = 'test_01_tfs_mm_fold_1.csv'
            filename_fold_2 = 'test_01_tfs_mm_fold_2.csv'
        else:
            filename_fold_1 = 'test_02_poin_mm_fold_1.csv'
            filename_fold_2 = 'test_02_poin_mm_fold_2.csv'

        df_fold_1 = pd.read_csv(os.path.join(path_ckpt,filename_fold_1))
        df_fold_2 = pd.read_csv(os.path.join(path_ckpt,filename_fold_2))

        print('df_fold_1.shape',df_fold_1.shape)
        print('df_fold_2.shape',df_fold_2.shape)
        df_fold_1_2_tfs = df_fold_1.merge(df_fold_2, how='outer', on='table_file_name', indicator = True)
        # Fold-1 and 2 combined results:
        # df_fold_1_results = df_fold_1_2_tfs.loc[(df_fold_1_2_tfs['_merge'] == 'left_only')]
        # df_fold_2_results = df_fold_1_2_tfs.loc[(df_fold_1_2_tfs['_merge'] == 'right_only')]

        y_pred_1 = df_fold_1_2_tfs['y_pred_x'].to_list()
        y_true_1 = df_fold_1_2_tfs['y_true_x'].to_list()
        y_pred_2 = df_fold_1_2_tfs['y_pred_y'].to_list()
        y_true_2 = df_fold_1_2_tfs['y_true_y'].to_list()

        df_fold_1 = pd.DataFrame({'table_file_name':df_fold_1_2_tfs['table_file_name'].to_list(),'y_pred':y_pred_1,'y_true':y_true_1})
        df_fold_2 = pd.DataFrame({'table_file_name':df_fold_1_2_tfs['table_file_name'].to_list(),'y_pred':y_pred_2,'y_true':y_true_2})

        # Remove SVT class:
        df_fold_1 = df_fold_1.loc[(df_fold_1['y_true'] == 0) | \
                            (df_fold_1['y_true'] == 1) | \
                            (df_fold_1['y_true'] == 2)] # Dong, 07/01/2024.
        df_fold_2 = df_fold_2.loc[(df_fold_2['y_true'] == 0) | \
                            (df_fold_2['y_true'] == 1) | \
                            (df_fold_2['y_true'] == 2)] # Dong, 07/01/2024.
        # Since 'y_pred_prob_0', 'y_pred_prob_1', 'y_pred_prob_2' are not in the table, I will fake one until I have them.
        df_fold_1.insert(3,'y_pred_prob_0',np.zeros((df_fold_1.shape[0],),dtype=float))
        df_fold_1.insert(3,'y_pred_prob_1',np.zeros((df_fold_1.shape[0],),dtype=float))
        df_fold_1.insert(3,'y_pred_prob_2',np.zeros((df_fold_1.shape[0],),dtype=float))
        df_fold_2.insert(3,'y_pred_prob_0',np.zeros((df_fold_2.shape[0],),dtype=float))
        df_fold_2.insert(3,'y_pred_prob_1',np.zeros((df_fold_2.shape[0],),dtype=float))
        df_fold_2.insert(3,'y_pred_prob_2',np.zeros((df_fold_2.shape[0],),dtype=float))

        print('df_fold_1[y_true] == 3',df_fold_1.loc[df_fold_1['y_true'] == 3])
    elif str_model_idx == '18' or str_model_idx == '19':
        fold_name = 'fold_1'
        filename_ckpt = 'test_02_Liu_JAHA_2022_the_other_fold_train_RNN_GRU_'+file_date+'_'+fold_name+'.pt'
        df_fold_1_fold = my_cfmatrix.my_load_pt_results(path_ckpt, filename_ckpt)

        fold_name = 'fold_2'
        filename_ckpt = 'test_02_Liu_JAHA_2022_the_other_fold_train_RNN_GRU_'+file_date+'_'+fold_name+'.pt'
        df_fold_2_fold = my_cfmatrix.my_load_pt_results(path_ckpt, filename_ckpt)

        fold_name = 'fold_1'
        filename_ckpt = 'test_03_Liu_JAHA_2022_remain_train_RNN_GRU_'+file_date+'_'+fold_name+'.pt'
        df_fold_1_NSR = my_cfmatrix.my_load_pt_results(path_ckpt, filename_ckpt)

        fold_name = 'fold_2'
        filename_ckpt = 'test_03_Liu_JAHA_2022_remain_train_RNN_GRU_'+file_date+'_'+fold_name+'.pt'
        df_fold_2_NSR = my_cfmatrix.my_load_pt_results(path_ckpt, filename_ckpt)

        path_output = '/'.join(path_ckpt.split('/')[:-1])
        df_fold_1 = pd.concat([df_fold_1_fold, df_fold_1_NSR], ignore_index=True, sort=False)
        df_fold_2 = pd.concat([df_fold_2_fold, df_fold_2_NSR], ignore_index=True, sort=False)
        df_fold_1_2_tfs = df_fold_1.merge(df_fold_2, how='outer', on='table_file_name', indicator = True)
    elif str_model_idx == '20':
        # Darren said on 09/28/2024 that he has same files like I have in
        # R:\ENGR_Chon\Darren\Honors_Thesis\saves_tfs\cassey_fold_1_trained\output_files
        y_pred_filename_fold_1 = 'predictions_fold_1_other_fold_test_set.csv'
        y_pred_filename_fold_2 = 'predictions_fold_2_other_fold_test_set.csv'

        y_true_filename_fold_1 = 'labels_fold_1_other_fold_test_set.csv'
        y_true_filename_fold_2 = 'labels_fold_2_other_fold_test_set.csv'

        pred_prob_filename_fold_1 = 'prediction_proba_fold_1_other_fold_test_set.csv'
        pred_prob_filename_fold_2 = 'prediction_proba_fold_2_other_fold_test_set.csv'

        y_pred_fold_1 = pd.read_csv(os.path.join(path_ckpt,y_pred_filename_fold_1))
        y_pred_fold_2 = pd.read_csv(os.path.join(path_ckpt,y_pred_filename_fold_2))
        y_true_fold_1 = pd.read_csv(os.path.join(path_ckpt,y_true_filename_fold_1))
        y_true_fold_2 = pd.read_csv(os.path.join(path_ckpt,y_true_filename_fold_2))
        pred_prob_fold_1 = pd.read_csv(os.path.join(path_ckpt,pred_prob_filename_fold_1))
        pred_prob_fold_2 = pd.read_csv(os.path.join(path_ckpt,pred_prob_filename_fold_2))

        # print('y_pred_fold_1.columns',y_pred_fold_1.columns) # 'predictions'
        # print('y_true_fold_1.columns',y_true_fold_1.columns) # 'labels'
        # print('pred_prob_fold_1.columns',pred_prob_fold_1.columns) # '0', '1', '2'

        df_fold_1_fold = pd.DataFrame({'y_pred':y_pred_fold_1['predictions'].to_list(),
                                  'y_true':y_true_fold_1['labels'].to_list(),
                                  'y_pred_prob_0':pred_prob_fold_1['0'].to_list(),
                                  'y_pred_prob_1':pred_prob_fold_1['1'].to_list(),
                                  'y_pred_prob_2':pred_prob_fold_1['2'].to_list()})
        
        # print('df_fold_1',df_fold_1)

        df_fold_2_fold = pd.DataFrame({'y_pred':y_pred_fold_2['predictions'].to_list(),
                                  'y_true':y_true_fold_2['labels'].to_list(),
                                  'y_pred_prob_0':pred_prob_fold_2['0'].to_list(),
                                  'y_pred_prob_1':pred_prob_fold_2['1'].to_list(),
                                  'y_pred_prob_2':pred_prob_fold_2['2'].to_list()})
        
        # print('df_fold_2',df_fold_2)
        y_pred_filename_fold_1 = 'predictions_fold_1_remove_both_folds.csv'
        y_pred_filename_fold_2 = 'predictions_fold_2_remove_both_folds.csv'

        y_true_filename_fold_1 = 'labels_fold_1_remove_both_folds.csv'
        y_true_filename_fold_2 = 'labels_fold_2_remove_both_folds.csv'

        pred_prob_filename_fold_1 = 'prediction_proba_fold_1_remove_both_folds.csv'
        pred_prob_filename_fold_2 = 'prediction_proba_fold_2_remove_both_folds.csv'

        y_pred_fold_1 = pd.read_csv(os.path.join(path_ckpt,y_pred_filename_fold_1))
        y_pred_fold_2 = pd.read_csv(os.path.join(path_ckpt,y_pred_filename_fold_2))
        y_true_fold_1 = pd.read_csv(os.path.join(path_ckpt,y_true_filename_fold_1))
        y_true_fold_2 = pd.read_csv(os.path.join(path_ckpt,y_true_filename_fold_2))
        pred_prob_fold_1 = pd.read_csv(os.path.join(path_ckpt,pred_prob_filename_fold_1))
        pred_prob_fold_2 = pd.read_csv(os.path.join(path_ckpt,pred_prob_filename_fold_2))

        # print('y_pred_fold_1.columns',y_pred_fold_1.columns) # 'predictions'
        # print('y_true_fold_1.columns',y_true_fold_1.columns) # 'labels'
        # print('pred_prob_fold_1.columns',pred_prob_fold_1.columns) # '0', '1', '2'

        df_fold_1_NSR = pd.DataFrame({'y_pred':y_pred_fold_1['predictions'].to_list(),
                                  'y_true':y_true_fold_1['labels'].to_list(),
                                  'y_pred_prob_0':pred_prob_fold_1['0'].to_list(),
                                  'y_pred_prob_1':pred_prob_fold_1['1'].to_list(),
                                  'y_pred_prob_2':pred_prob_fold_1['2'].to_list()})
        
        # print('df_fold_1',df_fold_1)

        df_fold_2_NSR = pd.DataFrame({'y_pred':y_pred_fold_2['predictions'].to_list(),
                                  'y_true':y_true_fold_2['labels'].to_list(),
                                  'y_pred_prob_0':pred_prob_fold_2['0'].to_list(),
                                  'y_pred_prob_1':pred_prob_fold_2['1'].to_list(),
                                  'y_pred_prob_2':pred_prob_fold_2['2'].to_list()})
        
        # print('df_fold_2',df_fold_2)
        df_fold_1 = pd.concat([df_fold_1_fold, df_fold_1_NSR], ignore_index=True, sort=False)
        df_fold_2 = pd.concat([df_fold_2_fold, df_fold_2_NSR], ignore_index=True, sort=False)
        # df_fold_1_2_tfs = df_fold_1.merge(df_fold_2, how='outer', on='table_file_name', indicator = True)
    else:
        fold_name = 'fold_1'
        filename_ckpt = 'test_02_the_other_fold_train_RNN_GRU_'+file_date+'_'+fold_name+'.pt'
        df_fold_1_fold = my_cfmatrix.my_load_pt_results(path_ckpt, filename_ckpt)

        fold_name = 'fold_2'
        filename_ckpt = 'test_02_the_other_fold_train_RNN_GRU_'+file_date+'_'+fold_name+'.pt'
        df_fold_2_fold = my_cfmatrix.my_load_pt_results(path_ckpt, filename_ckpt)

        path_output = '/'.join(path_ckpt.split('/')[:-1])

        fold_name = 'fold_1'
        filename_ckpt = 'test_03_remain_train_RNN_GRU_'+file_date+'_'+fold_name+'.pt'
        df_fold_1_NSR = my_cfmatrix.my_load_pt_results(path_ckpt, filename_ckpt)

        fold_name = 'fold_2'
        filename_ckpt = 'test_03_remain_train_RNN_GRU_'+file_date+'_'+fold_name+'.pt'
        df_fold_2_NSR = my_cfmatrix.my_load_pt_results(path_ckpt, filename_ckpt)

        path_output = '/'.join(path_ckpt.split('/')[:-1])

        df_fold_1 = pd.concat([df_fold_1_fold, df_fold_1_NSR], ignore_index=True, sort=False)
        df_fold_2 = pd.concat([df_fold_2_fold, df_fold_2_NSR], ignore_index=True, sort=False)

    # print('model',str_model_idx,'df_fold_1.shape',df_fold_1.shape,'df_fold_2.shape',df_fold_2.shape)
    df_all = pd.concat([df_fold_1, df_fold_2], ignore_index=True, sort=False)
    # Calculate Sensitivity (TPR), Specificity (TNR), Precision (PPV), NPV, FPR, FNR, FDR, Accuracy (ACC).
    dict_metrics_all = my_cfmatrix.my_average_none_metrics(y_true = df_all['y_true'], y_pred = df_all['y_pred'])

    rng_seed = 42
    rng = np.random.RandomState(rng_seed)
    n_bootstraps = 1000
    micro_aucs = []
    macro_aucs = []
    # Calculate the AUROC.
    n_classes = 3
    my_class_names = {0: 'NSR', 1: 'AF', 2: 'PAC/PVC'}
    # Get the class ratio for the entire DataFrame
    from functools import partial
    vc_norm = partial(pd.Series.value_counts, normalize=True)
    # print('df_all y_true value_counts')
    # print(df_all['y_true'].agg([pd.Series.value_counts, vc_norm]))

    # print(df_value_counts)
    for i in range(n_bootstraps):
        
        # bootstrap by sampling with replacement on the prediction indices
        sample_n = rng.randint(0, df_all.shape[0], size=1)

        # sample_n = len(indices)

        # df_all_sample = df_all.sample(n=sample_n, random_state=rng_seed)
        # df_all_sample = df_all.groupby('y_true',group_keys = False).apply(lambda x: x.sample(n=sample_n, frac=None, replace=True, random_state=rng_seed),include_groups = True)
        df_all_sample = df_all.groupby('y_true', group_keys=False).apply(lambda x: x.sample(int(np.rint(sample_n*len(x)/len(df_all))))).sample(frac=1).reset_index(drop=True)

        # print(f'In CI {i}/{n_bootstraps}, len(df_all_sample)={len(df_all_sample)}')
        unique_class = len(df_all_sample['y_true'].unique())#len(np.unique(y_true[indices]))
        if unique_class < n_classes:
            # We need at least one positive and one negative sample for ROC AUC
            # to be defined: reject the sample
            continue

        # Get the ratio of class for the sampled DataFrame
        vc_norm = partial(pd.Series.value_counts, normalize=True)
        # print('df_all_sample y_true value_counts')
        # print(df_all_sample['y_true'].agg([pd.Series.value_counts, vc_norm]))

        test_name = 'AUROC_all_pulsewatch_subject_independent_test_CI'
        shorted_test_name = 'CI'
        path_output = os.path.join(r'/mnt/r/ENGR_Chon/Dong/Python_generated_results/deep_learning_2023/analysis_2024_07_30',test_name)
        path_output_fig = os.path.join(path_output,'plots')
        pathlib.Path(path_output_fig).mkdir(parents=True, exist_ok=True)
        micro_roc_auc_ovr, macro_roc_auc_ovr = my_cfmatrix.my_auroc_cal_plot(df_all_sample, n_classes, my_class_names, path_output_fig, model_name, test_name, str_model_idx, fold_name='all')
        micro_aucs.append(micro_roc_auc_ovr)
        macro_aucs.append(macro_roc_auc_ovr)
    
    # Calculate the AUROC condifence interval.
    alpha = 0.05
    micro_lower_bound = np.percentile(micro_aucs, (1 - alpha) / 2 * 100)
    micro_upper_bound = np.percentile(micro_aucs, (1 + alpha) / 2 * 100)

    macro_lower_bound = np.percentile(macro_aucs, (1 - alpha) / 2 * 100)
    macro_upper_bound = np.percentile(macro_aucs, (1 + alpha) / 2 * 100)

    print('micro_lower_bound',micro_lower_bound,'micro_upper_bound',micro_upper_bound)
    print('macro_lower_bound',macro_lower_bound,'macro_upper_bound',macro_upper_bound)
    
    # Add new key value pair
    dict_metrics_all['micro_auroc'] = np.mean(micro_aucs)
    dict_metrics_all['macro_auroc'] = np.mean(macro_aucs)
    dict_metrics_all['micro_lower_bound'] = micro_lower_bound
    dict_metrics_all['micro_upper_bound'] = micro_upper_bound
    dict_metrics_all['macro_lower_bound'] = macro_lower_bound
    dict_metrics_all['macro_upper_bound'] = macro_upper_bound

    # df_metrics = pd.DataFrame(dict_metrics_avg.items(),columns=dict_metrics_avg.keys())
    df_metrics = pd.DataFrame.from_dict(dict_metrics_all, orient='columns').reset_index()
    df_metrics.to_csv(os.path.join(path_output,str_model_idx+'_'+model_name+'_'+shorted_test_name+'.csv'),header=True,index=True)

/home/doh16101/anaconda3/envs/CS330_torch/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


str_model_idx 03
Inside Linux
Not inside Colab
[[166825   5023   8053]
 [   241  22014   2300]
 [  1390   2852   8797]]
TP [166825.  22014.   8797.]
FN [13076.  2541.  4242.]
micro_lower_bound 0.9789060598389742 micro_upper_bound 0.9789313493053761
macro_lower_bound 0.9583617412204993 macro_upper_bound 0.958415160290333
str_model_idx 04
Inside Linux
Not inside Colab
[[160659  12221   7021]
 [   540  20033   3982]
 [  1494   3088   8457]]
TP [160659.  20033.   8457.]
FN [19242.  4522.  4582.]
micro_lower_bound 0.9543424250424714 micro_upper_bound 0.9543753761159395
macro_lower_bound 0.9383716804844981 macro_upper_bound 0.9384281076860351
str_model_idx 05
Inside Linux
Not inside Colab
[[168002   2472   9427]
 [   354  22465   1736]
 [  1760   2041   9238]]
TP [168002.  22465.   9238.]
FN [11899.  2090.  3801.]
micro_lower_bound 0.9811043210905362 micro_upper_bound 0.9811205453516412
macro_lower_bound 0.9514144373489178 macro_upper_bound 0.9514794683590697
str_model_idx 06
Inside Linux
No

In [ ]:
micro_aucs